In [1]:
# first set global root data directory
import pylabs
pylabs.datadir.target = 'jaba'
from pathlib import *
import pandas as pd
import numpy as np
from pylabs.io.mixed import df2h5, h52df, get_h5_keys
from pylabs.utils import *
from pylabs.projects.genz.file_names import project, Optsd
import qgrid
qgrid.enable
import ipywidgets as widgets
fs = Path(getnetworkdataroot())
opts = Optsd()
style = {'description_width': 'initial'}
input_selkey = widgets.Dropdown(options=sorted(get_h5_keys(opts.info_fname, key='auto_qc')), disabled=False,
    description='Subject DWI to qc:', style=style)
input_selkey.layout.width = '350px'
input_selkey

/home/toddr/Software/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


setting root data directory to jaba.
setting root data directory to jaba.
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
/brainstudio/data
setting root data directory to jaba.
setting root data directory to jaba.


Dropdown(description=u'Subject DWI to qc:', layout=Layout(width=u'350px'), options=('/sub-genz103/ses-1/dwi/auto_qc', '/sub-genz104/ses-1/dwi/auto_qc', '/sub-genz201/ses-1/dwi/auto_qc', '/sub-genz203/ses-1/dwi/auto_qc', '/sub-genz205/ses-1/dwi/auto_qc', '/sub-genz301/ses-1/dwi/auto_qc', '/sub-genz302/ses-1/dwi/auto_qc', '/sub-genz303/ses-1/dwi/auto_qc', '/sub-genz305/ses-1/dwi/auto_qc', '/sub-genz308/ses-1/dwi/auto_qc', '/sub-genz311/ses-1/dwi/auto_qc', '/sub-genz501/ses-1/dwi/auto_qc', '/sub-genz502/ses-1/dwi/auto_qc', '/sub-genz503/ses-1/dwi/auto_qc', '/sub-genz506/ses-1/dwi/auto_qc', '/sub-genz508/ses-1/dwi/auto_qc', '/sub-genz510/ses-1/dwi/auto_qc', '/sub-genz906/ses-1/dwi/auto_qc'), style=DescriptionStyle(description_width=u'initial'), value='/sub-genz103/ses-1/dwi/auto_qc')

In [2]:
pick = {'subj': input_selkey.value.split('/')[1], 'session': input_selkey.value.split('/')[2], }
# new h52df function makes all cols numeric. yes!
all_dwi_auto_qc = h52df(opts.info_fname, input_selkey.value)
pick['dwi_fname'] = all_dwi_auto_qc.loc[0, 'dwi_fname']
# todo: make python scriptlet to load files into FSLeyes
# fslv_names = ' '.join([x + ' -b 1,200' for x in all_dwi_auto_qc.loc[0, ['dwi_fname', 'topup_fname', 'topdn_fname']].values])
# with WorkingContext(str(Path(all_dwi_auto_qc.loc[0, 'dwi_fname']).parent)):
#     with open('fslview_cmd.sh', 'w') as f:
#         f.write('fslview ' + fslv_names + ' &\n')

# limit to only active rows with mask
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] != -9999, 'itopup_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] != -9999, 'itopdn_visqc'] = True
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopup'] == -9999, 'itopup_visqc'] = False
all_dwi_auto_qc.loc[all_dwi_auto_qc['itopdn'] == -9999, 'itopdn_visqc'] = False
all_dwi_auto_qc.loc[:, 'dwi_visqc'] = True
with WorkingContext(str(Path(pick['dwi_fname']).parent/'qc')):  
    topup_gvolspng = open('{subj}_{session}_topup8b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topup_bvolspng = open('{subj}_{session}_topup8b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    topdn_gvolspng = open('{subj}_{session}_topdn7b0-qc_good_plot.png'.format(**pick), 'rb').read()
    topdn_bvolspng = open('{subj}_{session}_topdn7b0-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi800_gvolspng = open('{subj}_{session}_b800-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi800_bvolspng= open('{subj}_{session}_b800-qc_bad_plot.png'.format(**pick), 'rb').read()
    dwi2000_gvolspng = open('{subj}_{session}_b2000-qc_good_plot.png'.format(**pick), 'rb').read()
    dwi2000_bvolspng= open('{subj}_{session}_b2000-qc_bad_plot.png'.format(**pick), 'rb').read()

dwicols = [u'bvals', u'auto_dwi_vol_idx', u'auto_dwi_qc', 'dwi_visqc', u'x_bvec', u'y_bvec', u'z_bvec']
topudcols = [u'itopup', u'alltopup_idx', u'topup_qc', 'itopup_visqc', u'itopdn', u'topdn_qc', 'itopdn_visqc',]
dwi_auto_qc = all_dwi_auto_qc[dwicols]
topud_auto_qc = all_dwi_auto_qc[topudcols]
dwi_w = qgrid.show_grid(dwi_auto_qc, show_toolbar=False)
topupdn_w = qgrid.show_grid(topud_auto_qc, show_toolbar=False)

In [3]:
items = [widgets.Image(description='dwi b2000 bad vols', value=dwi2000_bvolspng,    format='png',    width=500,    height=600),
        widgets.Image(description='dwi b2000 good vols', value=dwi2000_gvolspng,    format='png',    width=500,    height=600),]
#items_layout = [widgets.Layout(flex-basis='50%', ), widgets.Layout(flex-basis='50%', widgets.Layout(flex-basis='100%',)]
box_layout = widgets.Layout(display='flex', align_items='stretch', border='solid', width='100%')
top_box = widgets.HBox(children=items)
bottom_box = widgets.HBox(children=[dwi_w])
dwi_box = widgets.Box([top_box, bottom_box], layout=box_layout)
dwi_box

Box(children=(HBox(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xbf\xbf\xbf\x1f\x1f\x1f___???\xdf\xdf\xdf\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xf5Uk\xc9\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\x9d\x89r\xa3\xbc\x12F/\x9b\x1e.d\xa9\xec\x99?\xdbdR\xe2\xfd\x1f\xe1\xa2\xbd\xb5\x80\xc1\x06l\xd9\xdf\xa9\x9a\x89\x8d\x81\x06\x0c\xc7R#\x89\xffu\x00\x00\x90\t\xff;\xf6\x06\x00\x00\xc0T ,\x00@6@X\x00\x80l\x80\xb0\x00\x00\xd9\x00a\x01\x00\xb2\x01\xc2\x02\x00d\x03\x84\x05\x00\xc8\x06\x08\x0b\x00\x90\r\x97#,\xce\xabco\xc2\xe6pCU\xd5\xa5\x9a\xd4\xf2\x90ZL\xae8O\xad\xa0\xa8\xabJ,\\\xf8\x93\xcb\xb6\xe9\'7\xe1\xe4\x14\x03+\xde\x8e\xca\x1d\x84p\x83\x0b\xee\x9f\x14\x89\xc357\xd4\x8e9X]M>\r/\xf1\x84\xdd\r\x84\xb5(E\xdd_\xc9\xc1u\xc1\xbc\xd3\x98\xads\x01\xa7W\xeb\x8b\x89\x89Is\x84\xd5\x92\x8b\xbdu\x93\xcb\xc6M\xde\xa9\xac\x95\x84U(c27\x85\xb5\xf2\xd0\xb3p\xce\xca\xdb\xd9\x86~.\xf7\x83\x98)q\xb8f\xb1s_\x99\\\xf3\xc4\xb5%O\xd8x\xbfK\xe1\xc0\x86~\x0fS\xa6d\x0b\x84\xb5 \xacI]\x17\x85w\x8a\x16+]\xc0C\xc2\xaa%r\xc3*\xb1Q\x85\x9aP\xdb\x8f\x94]\x13\xd7\x9a\xda\x9b\xfe\xeah*o\x97\nor\x1b.\x16\xb0\x8a\xb0Jk!\x1b\xbe0\x93\xc2\xcb\xb2\xb2;*7\x98\x9c\x04\xcc\xf9Z\x918\\\xb3\xd8\xb9\xaf\xfdQo\xda\xa9\xdeH\x9c\xb0\x89\xfd6\xa7\\\xc3\xe6L\xc9\x17\x08kA\xc4\x89\xd1\xb4m\xed\x9d\xed\xcc;\x8d\xd9:%\x8e\x81\xd5\x92}\x16\x17tC>\n\x16\x88\x97\x17\xab\xe4-S\xaf[\xb7K\xa5\xd8IU,\x91E\xad\x1du\xa75\xf6\xb7\x94\xa2i\xeb\xca\xf9\xa6\x90\x87^N\t6\x88n@\xe1\t\xad\x7fW{g\xc5\xc8\xe1\x9a\xc4\xce}\xedg`\x93\xd7\x16\x9f\xb0\x89\xfdn\xd4\x14b\xe2)S2\x06\xc2Z\x8e\xfe\x02\xa8\xe4\xd5"\xca&\xe6\'P\xfe\xf2\xbbY\xbcw\xcbE\x1eX-\xddg! R\x1c\xda)\xac\xc6+c0{\x01{\xab\xa9w^\xd6k\x08\xab1\x85\x85\xc2\xf8\x89\x99\x17m\xf45{\x1bP\xd0\x12U\xbf\x1a\xa1\\\'\xb8\x91\xc35\x89)\xc2\x9a\xbe\xb6\xf8\x84\x8d\xf7\xbb\xd0_\x92;\xe3\xa6L\xc9\x19\x08k9j{\xf23\x1d\xac\xd6Ex\xfd\xb9\xf7n\xb9\xb0\xc3\xab\xf5\xf6\xb9\xf4\xde\xed\x12V\x11\xd4\x89\xc4\x05,\n\'\xadW\x89\x12\xb5(:W\xcc\n\xc2";b\xb6\xc6\x1d\xfa&\xdc \xbf\x80K\x04+\xbf$\xcf`#\x87k\x12+\x0b+\xb1\xdf\x95\xd9ofN\x80)Sr\xe6\x8c\x84\xa5\xf3\x91\xfa\xc4%_\xb7z)\xfeg"\x8d\xdc\xe8zN@+~\xbfZzR\xa9\x15\xda*\x84\xfc@\xa5/m\x0c\x82\xbcV\xbcy]\xc2\xd7N\x1c\x14\x96\xbf\xf1q\xf0\xc4\x1c$\xd6$au^ib\x97\xb0\xaa\xb0n\xa5/\x97\xa0VS\xc7I#\x7fC\xe5\x8a\xd5\xa4p\xe7Z\xbb\n\xf5Ia\xed\xd7\xea\xb2@z\x9f[WR0\xbf\x0cnG\xcb:\x98}PXj5\xf4\xc0\x8d\x1c./\xaa\xddko?H\xa8\xf4vs\xfaE\xa5\x16\xed\x0b\xca\x05\x9d=8a\xe3\xfdf\x9c\xd6\xfb\x8a\x89S|t\x88\x91\x13\xff\xa48\x1ba\xb9|\xa4\xfa\xc5L\t\xcb\xccS\xc5W\xbd\xfd\xc8f\x83\x98Y\xa1)h\x88\x0f\xda \x86/\xac\xc6\xfeX\xeb\x9b\xd7e\xd1cO\x04\xff\xdd\xd8\xc6\xc7\xc1\xa39\xc8\xc2\x83\xab\r\xae\xc0\x82V\tv\x08\xab\x8c\xebz\xf2\x02.\x83\r`e\xc9FwE\xac\xb8\x19\xda9S\xadQ\x9b%\xb2,\x85\x0b5\xb8\xcf51\x89\xda\xeer\xa4\xb2\xe3\xedYK.Y\xa5^j\xdc\x91\xc3\xe5]\xf8\xdar\xc1~\xd8PC\xdbM\x84\x95\\\xb4\xe5<\x10\x96\x

In [4]:
items = [widgets.Image(description='topup bad vols', value=topup_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topup good vols', value=topup_gvolspng,    format='png',    width=400,    height=600),
        widgets.Image(description='topdn bad vols', value=topdn_bvolspng,    format='png',    width=400,    height=600),
        #widgets.Image(description='topdn good vols', value=topdn_gvolspng,    format='png',    width=400,    height=600),
]
box_layout = widgets.Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
topupdn_box = widgets.Box(children=items+[topupdn_w], layout=box_layout)
topupdn_box

Box(children=(Image(value='\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x04\xb0\x00\x00\x03 \x08\x03\x00\x00\x00\x04>k0\x00\x00\x015PLTE\xff\xff\xff\x9f\x9f\x9f\xdf\xdf\xdf___???\xbf\xbf\xbf\x1f\x1f\x1f\xbe\xbe\xbe\xa0\xff |\xff@\xcd\xc0\xb0\xc1\xff\xc1\xa0\xb6\xcd\xf0\xff\xf0\xcd\xb7\x9e\xff\xff\xc0\xff\xff\x80\xff\xc0\xc0\xff\xa0p\xff\xa0`\xff\xa0@\xff\x80@\x80\x80\x00\x80`\xff\x80`\xc0\x80@\x80\x80@\x14\x80\x14\x14\x80\x14\x00Uk/\x90P@\xdd\xa0\xdd\x94\x00\xd3\xee\x82\xee\xff\xa5\x00\xa0\x80 \xf5\xf5\xdc\xb8\x86\x0b\xbd\xb7k\xf0\xe6\x8c\xe9\x96z\xfa\x80r\xffE\x00\xf0\x80\x80\xff\x7fP\xff\x14\x93\x00\xce\xd1\xff\x00\xff\x00\xff\xff\x87\xce\xeb\x00\x00\xcd\x00\x00\x80\x19\x19p\x00\x00\x8b\x00\x00\xff.\x8bW"\x8b"\x00\xff\x7f\x00d\x00\x00\xff\x00\xff\xd7\x00\xaf\xee\xee\xff\xb6\xc1\xee\xdd\x82\xe0\xff\xff\xf0U\xf0\xad\xd8\xe6\x90\xee\x90\xf022\xff\xff\xff\xe5\xe5\xe5\xcc\xcc\xcc\xc0\xc0\xc0\xb3\xb3\xb3\x99\x99\x99\x7f\x7f\x7ffffMMM333\x1a\x1a\x1a\x00\x00\x00@\xe0\xd0\xff\xff\x00\xa5**\x7f\xff\xd4\xff\x80\xff@\x80\x00\x8b\x00\x000`\x80\xc0\x80\xff\x00\x80@\xff\xc0 Ai\xe1\xc8\xc8\x00\xc0@\x00\x00\xee\xee\xc0\x00\xff\x00\x80\xff\x00\xc0\x00\xff\x00\x00\xa0\xa0\xa0\x00\x00\x00\xff\xff\xff\xa8*\xe1\x90\x00\x00\x00\x01tRNS\x00@\xe6\xd8f\x00\x00\x00\tpHYs\x00\x00\x0e\xc4\x00\x00\x0e\xc4\x01\x95+\x0e\x1b\x00\x00 \x00IDATx\x9c\xed\xdd\t\x96\xa2J\xb7\x06\xd0G\x17\x93c\x18\xff\xc2\xf9\x0f\xe1I\x13H\xa7)U&yO\xd6\xdek\xdd[\xa9\x02\x07P>!\x08\xf0\xffn\x00A\xfc\xdfO\xcf\x00\xc0\xbb\x04\x16\x10\x86\xc0\x02\xc2\x10X@\x18\x02\x0b\x08C`\x01a\x08, \x0c\x81\x05\x84\xf1\xab\x03\xab\xeb\xd2O\xcf\xc2\xe5\xba,\xa5\xaa\x19\x9fj\xbb\xad\xaa\x7f:u\xdd\xd1\x04\x8a*\xa5~\xe4b\xfdt\xd3\xd6\xf7\xa7\xeb\xed\xd3G\x9eL\xf8\xbb\xa4\xc7"og\xaf\xe8\xd6\x1f\x81\x83\x95s\xb6\xd4\x17C\x94UZU\xfca\xbfo\x0b\x10X\x7f\xab\xa8\xee[\xf2fK)W\x1f\xec\xf2{6\xe0\xe3\xc9\xae\x83\xa9\xec\x9f:\x13X\xedb\xf3o\x1fO7\xf5\xe3\xe9/#\xeb\x9b\x02\xab\xe9\xb3\xa0\xdeWO\xabE\xab\xcb\xc5K\xc3\\/\x92\xe9`\xe5\x9c\xf2\xe5\x92\x95\xc3\x94\xdf\x9cZ[U\xa7\xe7\xe0+MU\xf7\xb9\x9d\x97\xf9p\x0bx\xb6"C\x10X\x7f\xa7\xac\x8f\xb6\x94b\xf5\xa1-\xbeg\x03>\x9el\xbf%\x0e\x86\x19K\xfdL\x15\xe3\x13\xd5\xfc\xd2\x98\xae\x07[\xdf\xb84\xf7\x0f|\x9dV\x8bT\xac\x9en\xb7\xa3m|O`\xd5\x07\xeb9\x97\x9b\x16k\x98\xbd\xf4x\xa5|\xa4\xf3\xe8`\xe5\x9c\xf2\xe5\x92\xdd\xd7q\xdd\xbe\x1b\x05\x9f_O\xbbO\xe3\xd1\x16\xf0tE\x86 \xb0\xfeN\xff\xee\xd7m[\xad>\xff\xe5\xea\xa3X~\xcf\x06\xfcd\xb2\x8be.\xeeC\xd4\x8b\x976#\xec\xc7\xef\'\xd9\xb5\xe3b\x94\xedc\x91\x9a~!\xc7/\xedaW\xeb\x8b\xa3\xa9oY\xde\xben5\xae\xe7\x17\xe5\xfad-\x96\x8f\xaa\xd5\xf0/V\xce[\xbe\\\xb2\xfb\x00\xef\xc7\xc0\xc7\xd7\xd3\xf0\xfe\xddW\xd2\xf0\xa1\x1c\x9e9\xd8\x02\x9e\xaf\xc8\x10\x04\xd6_\xb9o\x12i\xd8|\xfb\xef\xb6\xbc\xe7\xd1o\x0b\x8bmh\xf5\xe8s\x95\x9fLv\xb9\xcc\xfd\x07x\xb1;\xf4e`\xd5\xab\xbd\x8er\xde\xa4W\x93\xa9\xbe\xdc\xd0\xbf#\xb0\x8ai\xd6\x96\xeb\xf9\xa8\\\xb1\xdc\xa3\xba\x0f\xdb\x07\xec#__\xac\x9c\xb7\xbc\x13X\x1f\x9c\xdaYU\xdem\x9a\x97m\xbf\x05\xbcX\x91!\x08\xac\xbfR\xcd\x9bC9\x15\xab\xa6&\x95\xe9\xf5\xd5\xa3\xcf\x95}>\xd9\xd527\xabG_\x05V\xd1\xad\x8f\x92\xfa\x8f}\xbf\xbb\xd2\xae\x0e\xab\xfa\xe3\xaa\xd7{\x11\xdf\x11X)\xaf\xe8r\xb7\xd8\xeb\xdd\xd9E\x9c\x0eo\xc9*\xc1^\xac\x9cw\xe7\xe2/\x07\xf8\xe3\x81\xdf\x9c\xe0\xf4\xce4\xd3j8\xdc\x1f}\xb6"C\x88\x1dX\xc5x\xe6j\xca\x8c\xc5\xbb3\xfe\xd9\xff\xbf\xec\x9b\x91\xeb\xf6p\x13k\xeb\xfe\xa5\xe5\'g\x9c\xe0|P1\xbc0\xb6Q\xce5\x16\x86\xef\xf0\xd5\xb0\x8f&\xe0\xf9\xc9\xa7\x81\xb5\x9e\xf9}\xf1\x83!\x16\xb5\xde\n\xac\xdbj\xff\xe2\xab\xc0J\xcb\x81{\xd3&\xbd9\xce\xa9\x96G]G3:Lx|j\xbbp\xed<\x89\xf1\x95bN\xbfv\xfa\xc2?^\xe6\xf2\xb1X\xf5\xb6\xfc\xd3\xc0\x1a\'\xb9\\M/V\xcec\x0e\x96\xcb\xb8\x9a\xebE\xa9\xe3\xb9\xec\x96o\xcb\xd1\xa8\xf7\xdd\xe2\xe2p\xe0\xed\xd0\xc3\x8c\x8e\x1f\xcfr7\xeb\xd3\x9f\xaf\x86)\x17\x81\xb5\xda\x02^\xad\xc8\xfe\xe5q\xd8\x17\x9b\xc4O\x8b\x1cX\xcd|~h\xfc\x0e=\n\xac<L\xdao\xf5\xf3KskP\x99\'\x98w4\xfa\x17\xdaM\x8du`\xd5\xf3\xd7w\x1a\xab7\xc5\xdd\xfcv\xaf\x1f\xbd\x9a\xf9}\xf1\xdd\x10\x8b\x91\x9fNv\xb3M\x16\xcb\xfd\xfe

In [7]:
dwi_w.get_changed_df()
topupdn_w.get_changed_df()
all_dwi_visqc = pd.concat([dwi_w.get_changed_df(), topupdn_w.get_changed_df()], axis=1)
df2h5(all_dwi_visqc, opts.info_fname, input_selkey.value.replace('auto_qc', 'vis_qc'))

In [6]:
topupdn_w.get_changed_df()

,itopup,alltopup_idx,topup_qc,itopup_visqc,itopdn,topdn_qc,itopdn_visqc
0,0,0,0,True,0,0,True
1,1,1,0,True,1,0,True
2,2,2,0,True,2,0,True
3,3,3,0,True,3,0,True
4,4,4,0,True,4,0,True
5,5,5,0,True,5,0,True
6,6,6,0,True,6,0,True
7,-9999,7,0,False,-9999,-9999,False
8,-9999,-9999,-9999,False,-9999,-9999,False
9,-9999,-9999,-9999,False,-9999,-9999,False


In [7]:
dwi_w.get_changed_df()

,bvals,auto_dwi_vol_idx,auto_dwi_qc,dwi_visqc,x_bvec,y_bvec,z_bvec
0,0,0,0,True,0.000,0.000,0.000
1,2000,0,1,True,1.000,0.000,0.000
2,2000,1,0,True,0.000,1.000,0.000
3,2000,2,0,True,0.000,0.000,1.000
4,2000,3,0,True,-0.625,-0.777,0.076
5,2000,4,0,True,-0.745,0.623,0.237
6,2000,5,1,True,0.025,-0.095,0.995
7,2000,6,0,True,-0.853,-0.004,-0.522
8,2000,7,1,True,-0.155,0.904,-0.399
9,2000,8,1,True,-0.025,-0.716,-0.698
